In [27]:

interval=10 #seconds
filepath = "../assets/place2017" #No trailing /

In [2]:
import datetime
import sys
import random
import gzip
import json
from tqdm import tqdm
import pickle

In [2]:
data = []
with open('place_tiles.csv', 'r') as f:
    f.readline()
    count=0
    #Useless but I'm adding it for fun
    nextcount=random.randrange(3000,20000)
    prevcount=0
    for line in f:
        count+=1
        if (prevcount+nextcount<=count):
            sys.stdout.write(f"\r{count} lines processed")
            sys.stdout.flush()
            prevcount=count
            nextcount=random.randrange(3000,20000)
        line = [x.rstrip() for x in line.split(",")]
        try:
            line[0]=datetime.datetime.strptime(line[0], '%Y-%m-%d %H:%M:%S.%f %Z')
        except:
            line[0]=datetime.datetime.strptime(line[0], '%Y-%m-%d %H:%M:%S %Z')
        if (line[2]=='' or line[3]=='' or line[4]==''):
            continue
        line[0]=int(line[0].timestamp()*1000)
        line[2]=int(line[2])
        line[3]=int(line[3])
        line[4]=int(line[4])
        data.append(line)
    sys.stdout.write(f"\r{count} lines processed")
    sys.stdout.flush()

16567567 lines processed

In [3]:
# import pickle
# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)


In [4]:
with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [5]:
print("Sorting...",end="")
data.sort()
print("Done")

Sorting...Done


In [6]:
with gzip.open('segments/1490172525.json.gz', 'rb') as f:
    file_content=f.read().decode('utf8')
hi = json.loads(file_content)
hi['events']

In [16]:
data=data[:100]

In [18]:
colors = ["#FFFFFF",
    "#E4E4E4",
    "#888888",
    "#222222",
    "#FFA7D1",
    "#E50000",
    "#E59500",
    "#A06A42",
    "#E5D900",
    "#94E044",
    "#02BE01",
    "#00E5F0",
    "#0083C7",
    "#0000EA",
    "#E04AFF",
    "#820080"
]

In [24]:
def package(field, arr, time):
    data = dict()
    data['field']=field
    temp=[]
    for i in range(len(arr)):
        cur=arr[i]
        temp.append({'id':i, 'timems': cur[0], 'user': cur[1], 'x': cur[2], 'y': cur[3], 'color': cur[4]})
    data['events']=temp
    data['sizex']=len(field)
    data['sizey']=len(field[0])
    data['basetime']=time
    file = gzip.open(f"{filepath}/segments/{time}f.json.gz", "wb")
    file.write(json.dumps(data).encode('utf8'))
    file.close()
    data['field']=None
    file = gzip.open(f"{filepath}/segments/{time}d.json.gz", "wb")
    file.write(json.dumps(data).encode('utf8'))
    file.close()

In [21]:
hardcodedstart=1490172525

In [25]:
metadata=dict()

In [30]:

field,log=[[[0,'Unchanged']]*1000]*1000,[]
prevfield=[[[0,'Unchanged']]*1000]*1000
chunk=data[0][0]//1000
nextchunk=chunk+interval
if hardcodedstart!=0: nextchunk=hardcodedstart
metadata['start']=nextchunk-interval
metadata['defaultslider']=nextchunk-interval
chunkcount=0
cont = True
for i in tqdm(range(0, len(data))):
    currow=data[i]
    ts,user,x,y,color=currow[0]//1000,currow[1],currow[2],currow[3],currow[4]
    while (currow[0]>nextchunk*1000):
        if chunkcount>50:
            cont=False
            break
        chunkcount+=1
        if chunkcount==1: chunk=nextchunk-interval
        package(prevfield, log, chunk)
        log=[]
        chunk=nextchunk
        nextchunk+=interval
        prevfield=field
    if not cont: break
    field[x][y]=[color, user]
    log.append(currow)

package(prevfield, log, chunk)
package(field, [], nextchunk)
metadata['end']=nextchunk
metadata['interval']=interval

100%|██████████| 100/100 [00:13<00:00,  7.31it/s]


In [31]:
metadata

{'start': 1490172515,
 'defaultslider': 1490172515,
 'end': 1490172855,
 'interval': 10}

In [ ]:
with open(f"{filepath}/metadata.json", "w") as f:
    f.write(json.dumps(metadata))